In [118]:
from __future__ import annotations

from bisect import bisect
from dataclasses import dataclass, field
import doctest
from functools import reduce

from utilities import extract_ints


In [142]:
@dataclass(frozen=True)
class Range:
  dst: int
  src: int
  length: int

  @classmethod
  def parse_from(cls, l: str) -> 'Range':
    """
    >>> Range.parse_from('50 98 2')
    Range(dst=50, src=98, length=2)
    """
    src, dst, length = [int(c) for c in l.strip().split()]
    return Range(src, dst, length)
  
  def map(self, src: int) -> int:
    """
    >>> r = Range(dst=50, src=98, length=2)
    >>> r.map(98)
    50
    >>> r.map(99)
    51
    >>> r.map(52)
    52
    >>> r.map(8)
    8
    """
    if self.contains(src):
      return self.dst + (src - self.src)
    else:
      return src

  def rev(self) -> 'Range':
    return Range(src=self.dst, dst=self.src, length=self.length)

  def contains(self, src: int) -> bool:
    return self.src <= src < (self.src + self.length)


@dataclass
class RangeMap:
  ranges: list[Range]
  srcs: list[int] = field(init=False)

  def __post_init__(self):
    self.ranges.sort(key=lambda r: r.src)
    self.srcs = [r.src for r in self.ranges]

  @classmethod
  def parse_from(cls, txt: str) -> RangeMap:
    """
    >>> RangeMap.parse_from('soil-to-seed map:\\n50 98 2\\n52 50 48')
    RangeMap(ranges=[Range(dst=52, src=50, length=48), Range(dst=50, src=98, length=2)], srcs=[50, 98])
    """
    return RangeMap(ranges=[Range.parse_from(l) for l in txt.strip().splitlines()[1:]])
    
  def map(self, src: int) -> int:
    """
    >>> rm = RangeMap([Range(dst=50, src=98, length=2), Range(dst=52, src=50, length=48)])
    >>> rm.map(79)
    81
    >>> rm.map(14)
    14
    >>> rm.map(55)
    57
    >>> rm.map(50)
    52
    >>> rm = RangeMap([Range(0, 15, 37), Range(37, 52, 2), Range(39, 0, 15)])
    >>> rm.map(14)
    53
    >>> rm.map(81)
    81
    """
    if (i := bisect(self.srcs, src)):
      return self.ranges[i-1].map(src)
    else:
      return src

  def rev(self) -> 'RangeMap':
    return RangeMap([r.rev() for r in self.ranges])
    
  def contains(self, src: int) -> bool:
    return any(r.contains(src) for r in self.ranges)
    

def parse_puzzle(txt: str):
  sections = txt.split('\n\n')
  seeds = extract_ints(sections[0])
  range_maps = [RangeMap.parse_from(s) for s in sections[1:]]
  return seeds, range_maps


apply_map = lambda src, rm: rm.map(src)
apply_maps = lambda maps, src: reduce(apply_map, maps, src)


def solution_1(seeds: list[int], range_maps: list[RangeMap]) -> int:
  return min(apply_maps(range_maps, s) for s in seeds)


def solution_2(seeds: list[int], range_maps: list[RangeMap]) -> int:
  seed_map = RangeMap([Range(seeds[i], seeds[i], seeds[i+1])
                       for i in range(0, len(seeds), 2)])
  rev_range_maps = [rm.rev() for rm in range_maps[::-1]]
  loc = 0
  while True:
    seed = reduce(apply_map, rev_range_maps, loc)
    if seed_map.contains(seed):
      return loc
    loc += 1


In [140]:
test_input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""


In [143]:
seeds, range_maps = parse_puzzle(test_input)
print('Part 1', solution_1(seeds, range_maps))
print('Part 2', solution_2(seeds, range_maps))


Part 1 35
Part 2 46


In [98]:
doctest.testmod(verbose=False, report=True, exclude_empty=True)


TestResults(failed=0, attempted=15)

In [144]:
# Final answers
with open('../data/day05.txt') as f:
    sections = f.read().split('\n\n')
    seeds = extract_ints(sections[0])
    range_maps = [RangeMap.parse_from(s) for s in sections[1:]]
    print('Part 1: ', solution_1(seeds, range_maps))
    print('Part 2: ', solution_2(seeds, range_maps))


Part 1:  340994526
Part 2:  52210644
